## Grover’s Search Algorithm

Grover’s algorithm is a quantum search procedure that efficiently finds a marked item within an unsorted set, providing a quadratic speedup over classical search (classical $O(N)$ vs. quantum $O(\sqrt{N})$. It works by amplifying the probability of the target state using repeated applications of the **oracle** (which marks the target by flipping its phase) and the **diffuser** (which performs inversion about the mean).  

![GroverAlgorithm](images/GroverAlgorithm.png)

- Grover’s algorithm begins by applying Hadamard gates (H) to all qubits, creating a uniform superposition where every basis state has equal probability.
- The Oracle (O) step inverts the amplitude of the marked state, effectively “tagging” the solution without changing its probability.
- The Diffuser (D) then performs an inversion about the mean, which boosts the amplitude of the marked state while suppressing all others.
- Repeating the Oracle and Diffuser together forms a **Grover iteration**, and each iteration further increases the probability of observing the marked state. The optimal number of iterations is given by  
  $k = \left\lfloor \tfrac{\pi}{4} \sqrt{\tfrac{N}{M}} \right\rfloor$,  
  where $N = 2^n$ is the total number of states and $M$ is the number of marked states.
- Finally, a measurement is performed. Because the amplitude of the marked state has been maximized, measuring the system yields the correct solution with high probability.

### Task
- Design and run a Grover’s search circuit for an $n$-qubit system, where $n$ is determined by the length of a user-specified binary target string (e.g., `1011`).  
- The oracle should flip the phase of the target state only, while the diffuser amplifies its probability amplitude through inversion about the mean.  
- For simplicity, construct both the oracle and the diffuser using multi-qubit controlled `mcx` gates.  
- Run the algorithm for the optimal number of iterations to maximize the probability of measuring the target state.  


### Expected Output
- A quantum circuit diagram of the Grover search algorithm showing the oracle, diffuser, and measurement steps.  
- A measurement histogram with probability mass concentrated on the target state.  

### Experimentation
- Modify the code to allow multiple marked states and adjust the number of Grover iterations accordingly.  
- Compare results for different numbers of iterations to observe under‑rotation and over‑rotation effects.


In [ ]:
from IPython.display import display
import numpy as np

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import circuit_drawer, plot_distribution
from qiskit_aer import AerSimulator

def grover_oracle(n, target):
    """
    Oracle that flips the phase of the target state.
    """
    qc = QuantumCircuit(n)
    target_bits = list(map(int, target))

    qc.barrier() # visual separation from previous circuit steps
    
    # Apply X to qubits where target bit is 0
    for i, bit in enumerate(target_bits):
        if bit == 0:
            qc.x(i)
    
    # Multi-controlled Z (phase flip)
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1)
    qc.h(n-1)
    
    # Uncompute the X gates
    for i, bit in enumerate(target_bits):
        if bit == 0:
            qc.x(i)
            
    qc.barrier() # visual separation from previous circuit steps 
    return qc

def grover_diffuser(n):
    """
    Diffuser: inversion about mean
    """
    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1)
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))
    return qc

def grover_search(target):
    """
    Builds a Grover search circuit for the given target bitstring.
    Applies Hadamards, the oracle, and the diffuser for the optimal 
    number of iterations, then measures.
    """
    n = len(target)
    x = QuantumRegister(n, name='x')
    c = ClassicalRegister(n, name='c')
    qc = QuantumCircuit(x, c)
    
    # Step 1: Initialize in uniform superposition
    qc.h(x)
    
    # Step 2: Number of Grover iterations
    N = 2**n
    num_iterations = int(np.floor(np.pi/4 * np.sqrt(N)))
    
    # Step 3: Grover iterations
    oracle = grover_oracle(n, target)
    diffuser = grover_diffuser(n)
    
    for _ in range(num_iterations):
        qc.compose(oracle, inplace=True)
        qc.compose(diffuser, inplace=True)
    
    # Step 4: Measure
    qc.measure(x, c)
    return qc

# ------------------------------------------------
#                main program
# ------------------------------------------------

target = input("Enter the 4-bit target state (e.g., 1011): ")
if len(target) < 2 or any(ch not in '01' for ch in target):
    raise ValueError("Target must be a binary string (at least 2 bits long) containing only 0 or 1.")
    
qc = grover_search(target)
display(circuit_drawer(qc, output="mpl"))

# Simulate
simulator = AerSimulator()
result = simulator.run(qc, shots=1024).result()
counts = result.get_counts(qc)

# Show results
distribution_plot = plot_distribution(counts)
display(distribution_plot)

# Get most frequent result and convert to big-endian (reverse the string)
most_frequent_le = max(counts, key=counts.get)
most_frequent_be = most_frequent_le[::-1]  # reverse the bitstring
print(f"The most probable solution: {most_frequent_be}")
